In [1]:
import  neo4j
import pandas as pd
import pyodbc

In [3]:


server = 'mcruebs04.isad.isadroot.ex.ac.uk'
database = 'BEMM459_GroupO'
username = 'GroupO'
password = 'MrjV827*Wr'

try:
    conn = pyodbc.connect(
        f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};',
        autocommit=True
    )
    print("Connected to Microsoft SQL Server!")
except Exception as e:
    print("Error connecting to SQL Server:", e)


Connected to Microsoft SQL Server!


In [4]:
cursor = conn.cursor()



# Create tables
tables = [
"""  
CREATE TABLE Customers (
    CustomerID INT PRIMARY KEY,
    Name VARCHAR(100),
    Email VARCHAR(100),
    Phone VARCHAR(20),
    Address VARCHAR(200),
    Type VARCHAR(50))
"""

"""
CREATE TABLE Parcels (
    ParcelID INT PRIMARY KEY,
    ParcelName VARCHAR(100),
    CustomerID INT,
    Weight FLOAT,
    Type VARCHAR(50),
    ShippingMethod VARCHAR(50),
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
)"""

 """
CREATE TABLE Shipments (
    ShipmentID INT PRIMARY KEY,
    ShipmentName VARCHAR(100),
    ParcelID INT,
    ShipmentDate DATETIME,
    DeliveryDate DATETIME,
    Status VARCHAR(50),
    CurrentLocation VARCHAR(100),
    FOREIGN KEY (ParcelID) REFERENCES Parcels(ParcelID)
)"""

"""
CREATE TABLE Personnel (
    PersonnelID INT PRIMARY KEY,
    Name VARCHAR(100),
    Phone VARCHAR(20),
    Role VARCHAR(50)
)"""

"""
CREATE TABLE ShipmentPersonnel (
    ShipmentID INT,
    PersonnelID INT,
    FOREIGN KEY (ShipmentID) REFERENCES Shipments(ShipmentID),
    FOREIGN KEY (PersonnelID) REFERENCES Personnel(PersonnelID)
);"""

"""
CREATE TABLE Analytics (
    ShipmentID INT,
    Delays INT,
    Efficiency FLOAT,
    CustomerRating INT,
    FOREIGN KEY (ShipmentID) REFERENCES Shipments(ShipmentID)
)"""
        
]

for table in tables:
    try:
        cursor.execute(table)
        print("Table created successfully!")
    except Exception as e:
        print("Error creating table:", e)

conn.commit()
cursor.close()
conn.close()


NameError: name 'conn' is not defined

In [5]:
from neo4j import GraphDatabase

# Neo4j connection details
URI = "bolt://localhost:7687"  # Update if using a different URI
USERNAME = "neo4j"              # Default username
PASSWORD = "12345678"           # Update with your actual password

# Connect to the Neo4j database
driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))

def create_node(tx, label, **properties):
    """
    Creates a node with the given label and properties.
    """
    query = f"CREATE (n:{label} {{{', '.join(f'{key}: ${key}' for key in properties.keys())}}})"
    tx.run(query, **properties)

def create_relationship(tx, label1, prop1, label2, prop2, relationship):
    """
    Creates a relationship between two nodes.
    """
    query = f"""
    MATCH (a:{label1} {{ID: $id1}}), (b:{label2} {{ID: $id2}})
    CREATE (a)-[:{relationship}]->(b)
    """
    tx.run(query, id1=prop1['ID'], id2=prop2['ID'])

# Main function to set up the database structure
def setup_database():
    with driver.session() as session:
        # Create nodes
        session.write_transaction(create_node, "Customer", ID="C1", Name="John Doe", Type="Individual", ContactInfo="john@example.com")
        session.write_transaction(create_node, "Parcel", ID="P1", Name="Electronics Box", Weight="5kg", Type="Fragile")
        session.write_transaction(create_node, "Shipment", ID="S1", Status="In Transit", Dates="2025-03-30", Location="Warehouse A")
        session.write_transaction(create_node, "Personnel", ID="PR1", Name="Alice", Role="Driver")
        session.write_transaction(create_node, "Analytics", ID="A1", Metrics="Delivery Time", Insights="On Time")

        # Create relationships
        session.write_transaction(create_relationship, "Customer", {"ID": "C1"}, "Parcel", {"ID": "P1"}, "OWNS")
        session.write_transaction(create_relationship, "Parcel", {"ID": "P1"}, "Shipment", {"ID": "S1"}, "SHIPPED_IN")
        session.write_transaction(create_relationship, "Shipment", {"ID": "S1"}, "Personnel", {"ID": "PR1"}, "HANDLED_BY")
        session.write_transaction(create_relationship, "Shipment", {"ID": "S1"}, "Analytics", {"ID": "A1"}, "HAS_ANALYTICS")

    print("Database structure set up successfully!")

# Run the setup
setup_database()

# Close the connection
driver.close()


C:\Users\bgane\AppData\Local\Temp\ipykernel_21092\1787676351.py:32: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node, "Customer", ID="C1", Name="John Doe", Type="Individual", ContactInfo="john@example.com")
C:\Users\bgane\AppData\Local\Temp\ipykernel_21092\1787676351.py:33: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node, "Parcel", ID="P1", Name="Electronics Box", Weight="5kg", Type="Fragile")
C:\Users\bgane\AppData\Local\Temp\ipykernel_21092\1787676351.py:34: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node, "Shipment", ID="S1", Status="In Transit", Dates="2025-03-30", Location="Warehouse A")
C:\Users\bgane\AppData\Local\Temp\ipykernel_21092\1787676351.py:35: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node, "Personnel", ID="PR1", Name=

Database structure set up successfully!


C:\Users\bgane\AppData\Local\Temp\ipykernel_21092\1787676351.py:42: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relationship, "Shipment", {"ID": "S1"}, "Analytics", {"ID": "A1"}, "HAS_ANALYTICS")


In [24]:
import pandas as pd
import pyodbc

# SQL Data Loader
class SQLDataLoader:
    def __init__(self, db_url):
        self.conn = pyodbc.connect(db_url)

    def load_data(self, file_path, sheet_name, table_name):
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        cursor = self.conn.cursor()
        for _, row in df.iterrows():
            placeholders = ', '.join(['?' for _ in row])
            query = f"INSERT INTO {table_name} VALUES ({placeholders})"
            cursor.execute(query, tuple(row))
        self.conn.commit()
        cursor.close()
        print(f"Data loaded into {table_name} successfully!")

# Usage 
sql_loader = SQLDataLoader("DRIVER={ODBC Driver 17 for SQL Server};SERVER=mcruebs04.isad.isadroot.ex.ac.uk;DATABASE=BEMM459_GroupO;UID=GroupO;PWD=MrjV827*Wr")



In [ ]:
# Load data into each table
sql_loader.load_data(r"C:\Users\bgane\Downloads\dhl\logistics_data.xlsx", "Customers", "Customers")
sql_loader.load_data(r"C:\Users\bgane\Downloads\dhl\logistics_data.xlsx", "Parcels", "Parcels")
sql_loader.load_data(r"C:\Users\bgane\Downloads\dhl\logistics_data.xlsx", "Shipments", "Shipments")
sql_loader.load_data(r"C:\Users\bgane\Downloads\dhl\logistics_data.xlsx", "Personnel", "Personnel")
sql_loader.load_data(r"C:\Users\bgane\Downloads\dhl\logistics_data.xlsx", "ShipmentPersonnel", "ShipmentPersonnel")
sql_loader.load_data(r"C:\Users\bgane\Downloads\dhl\logistics_data.xlsx", "Analytics", "Analytics")


In [18]:

cursor = sql_loader.conn.cursor()

tables = ["Customers", "Parcels", "Shipments", "Personnel", "ShipmentPersonnel", "Analytics"]

for table in tables:
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    count = cursor.fetchone()[0]
    print(f"Table {table} has {count} rows.")




for table in tables:
    cursor.execute(f"SELECT TOP 5 * FROM {table}")
    rows = cursor.fetchall()
    print(f"\nPreview of {table}:")
    for row in rows:
        print(row)
cursor.close()


Table Customers has 83 rows.
Table Parcels has 269 rows.
Table Shipments has 269 rows.
Table Personnel has 27 rows.
Table ShipmentPersonnel has 556 rows.
Table Analytics has 269 rows.

Preview of Customers:
(1, 'Customer_1', 'customer1@example.com', '+44 7126855092', 'Address_1', 'Premium')
(2, 'Customer_2', 'customer2@example.com', '+44 7362950628', 'Address_2', 'Regular')
(3, 'Customer_3', 'customer3@example.com', '+44 7249827706', 'Address_3', 'Regular')
(4, 'Customer_4', 'customer4@example.com', '+44 7826600539', 'Address_4', 'Regular')
(5, 'Customer_5', 'customer5@example.com', '+44 7734036506', 'Address_5', 'Premium')

Preview of Parcels:
(1, 'Parcel_1', 68, 18.41062314071619, 'Perishable', 'Express')
(2, 'Parcel_2', 3, 2.681420643757454, 'Perishable', 'Express')
(3, 'Parcel_3', 31, 1.629540746636657, 'Fragile', 'Standard')
(4, 'Parcel_4', 63, 16.41245344169172, 'Solid', 'Standard')
(5, 'Parcel_5', 61, 18.96354925424732, 'Solid', 'Express')

Preview of Shipments:
(1, 'Shipment_1'

In [20]:

cursor = sql_loader.conn.cursor()
#Function to get column names and data types
def get_column_info(table_name):
    query = f"""
    SELECT COLUMN_NAME, DATA_TYPE
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_NAME = '{table_name}';
    """
    cursor.execute(query)
    print(f"\nColumns for table '{table_name}':")
    for row in cursor.fetchall():
        print(f"Column Name: {row.COLUMN_NAME}, Data Type: {row.DATA_TYPE}")

# Loop through each table and print the column details
for table in tables:
    get_column_info(table)

# Close connection
cursor.close()


Columns for table 'Customers':
Column Name: CustomerID, Data Type: int
Column Name: Name, Data Type: varchar
Column Name: Email, Data Type: varchar
Column Name: Phone, Data Type: varchar
Column Name: Address, Data Type: varchar
Column Name: Type, Data Type: varchar

Columns for table 'Parcels':
Column Name: ParcelID, Data Type: int
Column Name: ParcelName, Data Type: varchar
Column Name: CustomerID, Data Type: int
Column Name: Weight, Data Type: float
Column Name: Type, Data Type: varchar
Column Name: ShippingMethod, Data Type: varchar

Columns for table 'Shipments':
Column Name: ShipmentID, Data Type: int
Column Name: ShipmentName, Data Type: varchar
Column Name: ParcelID, Data Type: int
Column Name: ShipmentDate, Data Type: datetime
Column Name: DeliveryDate, Data Type: datetime
Column Name: Status, Data Type: varchar
Column Name: CurrentLocation, Data Type: varchar

Columns for table 'Personnel':
Column Name: PersonnelID, Data Type: int
Column Name: Name, Data Type: varchar
Column 

In [25]:
import pandas as pd
from neo4j import GraphDatabase

# Connection details
URI = "bolt://localhost:7687"
USERNAME = "neo4j"
PASSWORD = "12345678"

driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))
# Load the Excel file
file_path = r"C:\Users\bgane\Downloads\dhl\logistics_data.xlsx"
excel_data = pd.ExcelFile(file_path)

# Function to create a node
def create_node(tx, label, **properties):
    query = f"MERGE (n:{label} {{{', '.join(f'{key}: ${key}' for key in properties.keys())}}})"
    tx.run(query, **properties)

# Function to create a relationship
def create_relationship(tx, label1, id1, label2, id2, relationship):
    query = f"""
    MATCH (a:{label1} {{ID: $id1}}), (b:{label2} {{ID: $id2}})
    MERGE (a)-[:{relationship}]->(b)
    """
    tx.run(query, id1=id1, id2=id2)

# Insert data from each sheet with relationships
def load_data():
    with driver.session() as session:
        # Load Customers
        customers = pd.read_excel(file_path, sheet_name='Customers')
        for _, row in customers.iterrows():
            session.write_transaction(create_node, "Customer", ID=str(row['CustomerID']), Name=row['Name'], ContactInfo=row['Email'], Type=row['Type'])

        # Load Personnel
        personnel = pd.read_excel(file_path, sheet_name='Personnel')
        for _, row in personnel.iterrows():
            session.write_transaction(create_node, "Personnel", ID=str(row['PersonnelID']), Name=row['Name'], Role=row['Role'])

        # Load Parcels and create OWNS relationship
        parcels = pd.read_excel(file_path, sheet_name='Parcels')
        for _, row in parcels.iterrows():
            session.write_transaction(create_node, "Parcel", ID=str(row['ParcelID']), Name=row['ParcelName'], Weight=row['Weight'], Type=row['Type'])
            session.write_transaction(create_relationship, "Customer", str(row['CustomerID']), "Parcel", str(row['ParcelID']), "OWNS")

        # Load Shipment Statuses as separate nodes
        shipments = pd.read_excel(file_path, sheet_name='Shipments')
        for _, row in shipments.iterrows():
            # Create a status node for each unique status
            session.write_transaction(create_node, "Status", ID=str(row['ShipmentID']), Name=row['Status'])
            # Create a relationship between Parcel and its Status
            session.write_transaction(create_relationship, "Parcel", str(row['ParcelID']), "Status", str(row['ShipmentID']), "DELIVERY_STATUS")
            # Create a shipment node as well
            session.write_transaction(create_node, "Shipment", ID=str(row['ShipmentID']), Name=row['ShipmentName'], Dates=str(row['ShipmentDate']), Location=row['CurrentLocation'])

        # Load Analytics and create HAS_ANALYTICS relationship
        analytics = pd.read_excel(file_path, sheet_name='Analytics')
        for _, row in analytics.iterrows():
            session.write_transaction(create_node, "Analytics", ID=str(row['ShipmentID']), Delays=row['Delays'], Efficiency=row['Efficiency'], CustomerRating=row['CustomerRating'])
            session.write_transaction(create_relationship, "Shipment", str(row['ShipmentID']), "Analytics", str(row['ShipmentID']), "HAS_ANALYTICS")

        # Load ShipmentPersonnel relationships
        shipment_personnel = pd.read_excel(file_path, sheet_name='ShipmentPersonnel')
        for _, row in shipment_personnel.iterrows():
            session.write_transaction(create_relationship, "Shipment", str(row['ShipmentID']), "Personnel", str(row['PersonnelID']), "HANDLED_BY")

    print("Data loaded successfully!")

# Run the data loading function
load_data()

driver.close()


c:\Users\bgane\anaconda3\Lib\mimetypes.py:99: RuntimeWarning: coroutine 'Server.serve' was never awaited
  exts = self.types_map_inv[strict].setdefault(type, [])
Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at c:\Users\bgane\anaconda3\Lib\site-packages\uvicorn\server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "c:\Users\bgane\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\bgane\AppData\Local\Temp\ipykernel_41240\803353980.py", line 60, in <module>
    server.run()
  File "c:\Users\bgane\anaconda3\Lib\site-packages\uvicorn\server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\bgane\anaconda3\Lib\site-packages\nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
      

Data loaded successfully!


Columns for table 'Customers':
Column Name: CustomerID, Data Type: int
Column Name: Name, Data Type: varchar
Column Name: Email, Data Type: varchar
Column Name: Phone, Data Type: varchar
Column Name: Address, Data Type: varchar
Column Name: Type, Data Type: varchar

Columns for table 'Parcels':
Column Name: ParcelID, Data Type: int
Column Name: ParcelName, Data Type: varchar
Column Name: CustomerID, Data Type: int
Column Name: Weight, Data Type: float
Column Name: Type, Data Type: varchar
Column Name: ShippingMethod, Data Type: varchar

Columns for table 'Shipments':
Column Name: ShipmentID, Data Type: int
Column Name: ShipmentName, Data Type: varchar
Column Name: ParcelID, Data Type: int
Column Name: ShipmentDate, Data Type: datetime
Column Name: DeliveryDate, Data Type: datetime
Column Name: Status, Data Type: varchar
Column Name: CurrentLocation, Data Type: varchar

Columns for table 'Personnel':
Column Name: PersonnelID, Data Type: int
Column Name: Name, Data Type: varchar
Column Name: Phone, Data Type: varchar
Column Name: Role, Data Type: varchar

Columns for table 'ShipmentPersonnel':
Column Name: ShipmentID, Data Type: int
Column Name: PersonnelID, Data Type: int

Columns for table 'Analytics':
Column Name: ShipmentID, Data Type: int
Column Name: Delays, Data Type: int
Column Name: Efficiency, Data Type: float
Column Name: CustomerRating, Data Type: int

In [26]:
from fastapi import FastAPI, HTTPException
from py2neo import Graph
import pyodbc

app = FastAPI()


cursor = sql_loader.conn.cursor()

# Neo4j Connection
graph = Graph("bolt://localhost:7687", auth=("neo4j", "12345678"))

# API to fetch customer details from SQL
@app.get("/customers/{customer_id}")
async def get_customer(customer_id: int):
    try:
        cursor.execute("SELECT * FROM Customers WHERE CustomerID = ?", (customer_id,))
        customer = cursor.fetchone()
        if customer:
            return {"CustomerID": customer[0], "Name": customer[1], "Address": customer[2]}
        else:
            raise HTTPException(status_code=404, detail="Customer not found")
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# API to fetch parcel details from SQL
@app.get("/parcels/{parcel_id}")
async def get_parcel(parcel_id: int):
    try:
        cursor.execute("SELECT * FROM Parcels WHERE ParcelID = ?", (parcel_id,))
        parcel = cursor.fetchone()
        if parcel:
            return {"ParcelID": parcel[0], "CustomerID": parcel[1], "Status": parcel[2]}
        else:
            raise HTTPException(status_code=404, detail="Parcel not found")
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


# API to update parcel status
@app.put("/parcels/{parcel_id}/status")
async def update_parcel_status(parcel_id: int, status: str):
    try:
        cursor.execute("UPDATE Parcels SET Status = ? WHERE ParcelID = ?", (status, parcel_id))
        #cursor.commit()
        return {"message": "Status updated successfully"}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Run the FastAPI app in Jupyter Notebook
import nest_asyncio
import uvicorn

# Apply the nest_asyncio patch to allow running uvicorn in Jupyter Notebook
nest_asyncio.apply()

# Run the FastAPI app
config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
server = uvicorn.Server(config)
server.run()



INFO:     Started server process [41240]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:60983 - "GET /customers/34 HTTP/1.1" 200 OK
INFO:     127.0.0.1:60985 - "GET /parcels/5 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [41240]


KeyboardInterrupt: 